In [5]:
from collections import Counter
import pandas as pd
import numpy as np
import os
import re

#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
remove_these = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jasmeen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
REMOVE_STOPWORDS = True

In [7]:
%%time

with open('lyrics/mxm_dataset_train.txt','r') as f:
    lines = f.readlines()
    words = lines[17].replace('%','').split(',')
    all_songs_dict = dict()
    for i,l in list(enumerate(lines))[18:]:
        song_info = l.split(',')
        MSDID = song_info[0]
        song_bow = [x.split(':') for x in song_info[2:]]
        song_dict = {}
        for word, word_count in song_bow:
            song_dict[int(word)] = int(word_count.replace('\n',''))
        word_lists = [[words[word-1]]*song_dict[word] for word in song_dict.keys()]
        song = [word for word_list in word_lists for word in word_list]
        if REMOVE_STOPWORDS:
            song = [w for w in song if w not in remove_these]
        all_songs_dict[str(MSDID)] = ' '.join(song).replace('\n','')

FileNotFoundError: [Errno 2] No such file or directory: 'lyrics/mxm_dataset_train.txt'

In [8]:
print(len(all_songs_dict.keys()))
song_msd_ids = list(all_songs_dict.keys())

NameError: name 'all_songs_dict' is not defined

In [ ]:
print(np.random.choice(song_msd_ids))

In [ ]:
%%time

all_song_meta_dict = dict()
with open('lyrics/mxm_779k_matches.txt','r') as f:
    lines = f.readlines()
    for i in range(18, len(lines)):
        line = lines[i].split('<SEP>')
        MSDID = line[0]
        artist = line[1]
        title = line[2]
        all_song_meta_dict[str(MSDID)] = {'artist': artist, 'title': title}

In [ ]:
len(all_song_meta_dict.keys())

In [ ]:
# all_artists = sorted(set([all_song_meta_dict[msdId]['artist'].lower() for msdId in all_song_meta_dict]))

In [ ]:
d = {
    'MSDID': list(all_songs_dict.keys()),
    'cleaned_text': [all_songs_dict[x] for x in all_songs_dict.keys()]
    }
msdid_df = pd.DataFrame.from_dict(d)
print(msdid_df.shape)
msdid_df.head()

In [ ]:
d = {
    'MSDID': msdid_df['MSDID'],
    'artist': [all_song_meta_dict[x]['artist'] for x in msdid_df['MSDID']],
    'title': [all_song_meta_dict[x]['title'] for x in msdid_df['MSDID']]
    }
meta_df = pd.DataFrame.from_dict(d)
print(meta_df.shape)
meta_df.head()

In [ ]:
meta_df
full_df_ = pd.merge(msdid_df, meta_df, on='MSDID', how='left')
print(full_df_.shape)
full_df_.head()

my_artists_ = ['Green Day','Blink-182','Sum 41','Rise Against',
              'Four Year Strong','Three Days Grace',
              'Tame Impala','Snoop Dogg','Thin Lizzy','The Strokes',
              'Nirvana','All Time Low', 'Foo Fighters','Lil Wayne',
              'Eminem','Tom Petty','Seether', 'Shinedown',
              'Red Hot Chili Peppers','The Who', 'The Doors',
              'Linkin Park','Arctic Monkeys','The Black Keys',
              'Led Zeppelin','The Kinks','Collective Soul',
              'Avenged Sevenfold','George Harrison','Eric Clapton',
              'The Donnas','System of a Down', 'MxPx',
              'Weezer','Rancid','Atmosphere','Santigold',
              'Iron Maiden','Fall Out Boy', 'Hank Williams']

my_artists = ['The Smashing Pumpkins', 'Bruce Springsteen','Beastie Boys']

sampled_artists = list(np.random.choice(full_df_['artist'], size=500, replace=False))
# sampled_artists

In [ ]:
# top_artists = full_df_['artist'].value_counts()[:10].reset_index()['index'].values
# full_df = full_df_[full_df_['artist'].isin(my_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists)].reset_index(drop=True)
# full_df = full_df_[full_df_['artist'].isin(sampled_artists + my_artists + my_artists_)].reset_index(drop=True)
full_df = full_df_.copy()
print(full_df.shape)
full_df.head()

# Train model and get embeddings

In [ ]:
%%time

full_df
# get training data
data = full_df['cleaned_text'].tolist()
# test_data = test_df['cleaned_text'].tolist()
train_corpus = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in list(enumerate(data))]
# build model
model = Doc2Vec(vector_size=50, min_count=1, epochs=10, dm=0)
model.build_vocab(train_corpus)
# train model
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.save('trained_model_example')

In [ ]:
model = Doc2Vec.load('trained_model_example')

In [ ]:
emb_df = pd.DataFrame([model.docvecs[f'{i}'] for i in range(len(full_df))])
fe_df = full_df.copy()
for c in emb_df.columns:
    fe_df[c] = emb_df[c]
# fe_df[fe_df['artist'].isin(my_artists)].groupby('artist').mean().reset_index()

In [ ]:
fe_df.head()

In [ ]:
fe_df.shape

In [ ]:
# merge the fe_df with spotify dataset on artist and title

spotify_df = pd.read_csv('spotify_data/train.csv')
spotify_df['artists'] = spotify_df['artists'].str.lower()
spotify_df['name'] = spotify_df['name'].str.lower()
spotify_df['artists'] = spotify_df['artists'].str.replace(' ','')
spotify_df['name'] = spotify_df['name'].str.replace(' ','')
spotify_df['artists'] = spotify_df['artists'].str.replace('.','')
spotify_df['name'] = spotify_df['name'].str.replace('.','')
spotify_df['artists'] = spotify_df['artists'].str.replace('\'','')
spotify_df['name'] = spotify_df['name'].str.replace('\'','')
spotify_df['artists'] = spotify_df['artists'].str.replace('[','')
spotify_df['name'] = spotify_df['name'].str.replace('[','')
spotify_df['artists'] = spotify_df['artists'].str.replace(']','')
spotify_df['name'] = spotify_df['name'].str.replace(']','')
spotify_df['artists'] = spotify_df['artists'].str.replace('-','')
spotify_df['name'] = spotify_df['name'].str.replace('-','')
spotify_df['artists'] = spotify_df['artists'].str.replace('\'','')
spotify_df['name'] = spotify_df['name'].str.replace('\'','')
spotify_df['artists'] = spotify_df['artists'].str.replace('!','')
spotify_df['name'] = spotify_df['name'].str.replace('!','')
spotify_df['artists'] = spotify_df['artists'].str.replace('(','')
spotify_df['name'] = spotify_df['name'].str.replace('(','')
spotify_df['artists'] = spotify_df['artists'].str.replace(')','')
spotify_df['name'] = spotify_df['name'].str.replace(')','')
spotify_df['artists'] = spotify_df['artists'].str.replace(',','')
spotify_df['name'] = spotify_df['name'].str.replace(',','')
spotify_df['artists'] = spotify_df['artists'].str.replace('&','')
spotify_df['name'] = spotify_df['name'].str.replace('&','')
spotify_df['artists'] = spotify_df['artists'].str.replace('/','')
spotify_df['name'] = spotify_df['name'].str.replace('/','')
spotify_df['artists'] = spotify_df['artists'].str.replace('feat','')
spotify_df['name'] = spotify_df['name'].str.replace('feat','')
spotify_df['artists'] = spotify_df['artists'].str.replace('featuring','')
spotify_df['name'] = spotify_df['name'].str.replace('featuring','')
spotify_df['artists'] = spotify_df['artists'].str.replace('ft','')
spotify_df['name'] = spotify_df['name'].str.replace('ft','')
spotify_df['artists'] = spotify_df['artists'].str.replace('ft.','')
spotify_df['name'] = spotify_df['name'].str.replace('ft.','')


#merge now 
fe_df['artist'] = fe_df['artist'].str.lower()
fe_df['title'] = fe_df['title'].str.lower()
fe_df['artist'] = fe_df['artist'].str.replace(' ','')
fe_df['title'] = fe_df['title'].str.replace(' ','')
fe_df['artist'] = fe_df['artist'].str.replace('.','')
fe_df['title'] = fe_df['title'].str.replace('.','')
fe_df['artist'] = fe_df['artist'].str.replace('\'','')
fe_df['title'] = fe_df['title'].str.replace('\'','')
fe_df['artist'] = fe_df['artist'].str.replace('[','')
fe_df['title'] = fe_df['title'].str.replace('[','')
fe_df['artist'] = fe_df['artist'].str.replace(']','')
fe_df['title'] = fe_df['title'].str.replace(']','')
fe_df['artist'] = fe_df['artist'].str.replace('-','')
fe_df['title'] = fe_df['title'].str.replace('-','')
fe_df['artist'] = fe_df['artist'].str.replace('\'','')
fe_df['title'] = fe_df['title'].str.replace('\'','')
fe_df['artist'] = fe_df['artist'].str.replace('!','')
fe_df['title'] = fe_df['title'].str.replace('!','')
fe_df['artist'] = fe_df['artist'].str.replace('(','')
fe_df['title'] = fe_df['title'].str.replace('(','')
fe_df['artist'] = fe_df['artist'].str.replace(')','')
fe_df['title'] = fe_df['title'].str.replace(')','')
fe_df['artist'] = fe_df['artist'].str.replace(',','')
fe_df['title'] = fe_df['title'].str.replace(',','')
fe_df['artist'] = fe_df['artist'].str.replace('&','')
fe_df['title'] = fe_df['title'].str.replace('&','')
fe_df['artist'] = fe_df['artist'].str.replace('/','')
fe_df['title'] = fe_df['title'].str.replace('/','')
fe_df['artist'] = fe_df['artist'].str.replace('feat','')
fe_df['title'] = fe_df['title'].str.replace('feat','')
fe_df['artist'] = fe_df['artist'].str.replace('featuring','')
fe_df['title'] = fe_df['title'].str.replace('featuring','')
fe_df['artist'] = fe_df['artist'].str.replace('ft','')
fe_df['title'] = fe_df['title'].str.replace('ft','')
fe_df['artist'] = fe_df['artist'].str.replace('ft.','')
fe_df['title'] = fe_df['title'].str.replace('ft.','')

# rename columns to match

spotify_df.rename(columns={'name':'title'}, inplace=True)
spotify_df.rename(columns={'artists':'artist'}, inplace=True)

In [ ]:
spotify_df

In [ ]:
# merge the two dataframes, while only showing the valid merges there should be no nulls
merged_df = pd.merge(fe_df, spotify_df, on=['artist','title'], how='inner')
merged_df

In [ ]:
# pring id feature of the merged dataframe
print(merged_df['id'])

In [ ]:
# merge the two dataframes, while only showing the valid merges there should be no nulls
spotify_orginal = pd.read_csv('spotify_data/train.csv')
# i wanna get the original spotify data merged into the new dataframe based on id column, only add name and artists columns
merged_df_newer = pd.merge(merged_df, spotify_orginal[['id','name','artists']], on='id', how='inner')
merged_df_newer.columns

In [ ]:
# make name and artists columns to be the frist and second columns
cols = ['id']
merged_df_newer = merged_df_newer[cols + [col for col in merged_df_newer.columns if col not in cols]]
merged_df_newer

In [ ]:
# drop MSID, artist and title column
merged_df_newer.drop(['MSDID','artist','title'], axis=1, inplace=True)
merged_df_newer

In [ ]:
merged_df_newer.to_csv('merged_df_newer.csv', index=False)

In [ ]:
merged_df_emb = merged_df_newer.copy()
merged_df_emb

In [ ]:
# # I wanna embed the song name and artist name in the dataframe using Word2Vec
merged_df_emb['artists'] = merged_df_emb['artists'].str.replace('[','')
merged_df_emb['name'] = merged_df_emb['name'].str.replace('[','')
merged_df_emb['artists'] = merged_df_emb['artists'].str.replace(']','')
merged_df_emb['name'] = merged_df_emb['name'].str.replace(']','')
merged_df_emb['artists'] = merged_df_emb['artists'].str.replace('\'','')
merged_df_emb['name'] = merged_df_emb['name'].str.replace('\'','')
merged_df_emb.head()

In [ ]:
# do a simple encoding on names and artists
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
merged_df_emb['artists'] = le.fit_transform(merged_df_emb['artists'])
merged_df_emb['name'] = le.fit_transform(merged_df_emb['name'])
merged_df_emb.head()

In [ ]:
clean_df = merged_df_emb.copy()
clean_df.drop(['cleaned_text'], axis=1, inplace=True)
clean_df.drop(['release_date'], axis=1, inplace=True)

In [ ]:
#print any feature that has string values left
for col in clean_df.columns:
    if clean_df[col].dtype == 'object':
        print(col)
    

In [ ]:
# make a new dataframe with columns 2 till end of the dataframe
clean_df = clean_df.iloc[:, 1:]
clean_df.head()

In [ ]:
#check if any row has string values
for col in clean_df.columns:
    if clean_df[col].dtype == 'object':
        print(col)
    

In [ ]:

non_categorical_columns = clean_df.columns
df_non_categorical = clean_df[non_categorical_columns]
norm_data = (df_non_categorical-df_non_categorical.min())/(df_non_categorical.max()-df_non_categorical.min())
norm_data.head()

In [ ]:
# add back id column 
norm_data['id'] = merged_df_emb['id']
norm_data


In [ ]:
norm_data.to_csv('norm_data.csv', index=False, header=True)



In [ ]:
norm_data_new = norm_data.copy()
norm_data_new.drop(['id'], axis=1, inplace=True)
norm_data_new

In [ ]:
norm_data_new.columns = norm_data_new.columns.astype(str)
norm_data_new.columns

# check if year has any string values
for col in norm_data_new.columns:
    if norm_data_new[col].dtype == 'object':
        print(col)

In [ ]:
# perform PCA on the data
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
principalComponents = pca.fit_transform(norm_data_new)
principalDf = pd.DataFrame(data = principalComponents)
principalDf.head()


In [ ]:
# perform SVD on the data
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=20)
svdComponents = svd.fit_transform(norm_data_new)
svdDf = pd.DataFrame(data = svdComponents)
svdDf.head()

In [ ]:
# # merge the PCA data with id column
# principalDf['id'] = norm_data['id']
# principalDf

In [ ]:
# # merge the SVD data with id column
# svdDf['id'] = norm_data['id']
# svdDf

In [ ]:
# make various plots and visualizations
import matplotlib.pyplot as plt

# plot the PCA data
plt.scatter(principalDf[0], principalDf[1])
plt.title('PCA')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.show()

# plot the SVD data
plt.scatter(svdDf[0], svdDf[1])
plt.title('SVD')
plt.xlabel('SVD 1')
plt.ylabel('SVD 2')
plt.show()


In [ ]:
# Calculate Explained Variance for svd and plot on my data
explained_variance = svd.explained_variance_ratio_
explained_variance

plt.plot(np.cumsum(explained_variance))
plt.title('Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Variance Explained')
plt.show()



In [ ]:
# Calculate Explained Variance: PCA
explained_variance_pca = pca.explained_variance_ratio_
explained_variance_pca

plt.plot(np.cumsum(explained_variance_pca))
plt.title('Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Variance Explained')
plt.show()


In [ ]:
def highly_correlated_features(df, threshold):
    correlation_matrix = df.corr() # Calculate the correlation matrix
    
    features_to_drop = set()
    highly_correlated_pairs = []
    
    for i in range(correlation_matrix.shape[0]):
        for j in range(i+1, correlation_matrix.shape[1]):
            # If the absolute value of the correlation coefficient is above the threshold, add the feature to the set
            if abs(correlation_matrix.iloc[i, j]) > threshold:
                features_to_drop.add(correlation_matrix.columns[j])
                highly_correlated_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

    return features_to_drop , highly_correlated_pairs

In [ ]:
drop_features, correlated_pairs  = highly_correlated_features(svdDf, 0.5)


print("We can drop these Features:",drop_features)
print()
print("Correlated Pairs and their coefficient")
for i in correlated_pairs:
    print(i)

None of the features are highly correlated

In [ ]:
# Compute clsuters to do unsupervised learning
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Create a list to store the silhouette scores
silhouette_scores = []

# Create a list of different number of clusters to try
n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# Loop through each cluster
for n in n_clusters:
    # Create a KMeans object with n clusters and fit it to the data
    kmeans = KMeans(n_clusters=n, random_state=0)
    kmeans.fit(svdDf)
    
    # Calculate the silhouette score and append it to the list
    silhouette_scores.append(silhouette_score(svdDf, kmeans.labels_))

# Plot the silhouette scores
plt.plot(n_clusters, silhouette_scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

In [ ]:
# also do elbow method
# Create a list to store the inertia values
inertia_values = []

# Loop through each cluster
for n in n_clusters:
    # Create a KMeans object with n clusters and fit it to the data
    kmeans = KMeans(n_clusters=n, random_state=0)
    kmeans.fit(svdDf)
    
    # Append the inertia to the list
    inertia_values.append(kmeans.inertia_)

# Plot the inertia values
plt.plot(n_clusters, inertia_values)
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Inertia vs Number of Clusters')
plt.show()


In [ ]:
# compute cosine similarity with all the songs
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrix
cosine_similarity_matrix_svd = cosine_similarity(svdDf)

# Create a DataFrame for the cosine similarity matrix
cosine_similarity_svd = pd.DataFrame(cosine_similarity_matrix_svd, index=svdDf.index, columns=svdDf.index)
cosine_similarity_svd.head()

In [ ]:
cosine_similarity_svd.to_csv('cosine_similarity_svd.csv', index=True)

In [ ]:
# compute cosine similarity with all the songs
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity matrix
cosine_similarity_matrix_pca = cosine_similarity(principalDf)

# Create a DataFrame for the cosine similarity matrix
cosine_similarity_pca = pd.DataFrame(cosine_similarity_matrix_pca, index=principalDf.index, columns=principalDf.index)
cosine_similarity_pca.head()

In [ ]:
# Create a function to get recommendations based on the cosine similarity matrix
def get_recommendations(song_title,artist_name ,cosine_similarity_df, data):
    # Get the index of the song based on title and artist
    song_index = data[(data['name'] == song_title) & (data['artists'] == artist_name)].index[0]
    # Get the similarity scores of the song
    similarity_scores = cosine_similarity_df[song_index]
    
    # Get the indices of the songs with the highest similarity scores
    similar_songs = similarity_scores.sort_values(ascending=False).index[1:4]
    
    # return like this: Index: 1, Name: 'song name', Artists: 'artist name'
    recommended_songs = []
    for i in similar_songs:
        recommended_songs.append({'Index': i, 'Name': data['name'][i], 'Artists': data['artists'][i]})
    
    return recommended_songs

In [ ]:
# Get recommendations for the song 'SLovelight' by ABBA
song, artist = 'Strawberry Swing','[\'Coldplay\']'
recommendations = get_recommendations(song, artist,cosine_similarity_svd, merged_df_newer)
print(f"Since you listened to {song}, you might also like:")
for i in recommendations:
    print('Index:', i['Index'], 'Name:', i['Name'], ' By:', i['Artists'])

In [ ]:
# I wanna do RMSE on the results 
from sklearn.metrics import mean_squared_error
from math import sqrt

# Create a function to calculate the RMSE
def calculate_rmse(predictions, actual):
    # Calculate the RMSE
    rmse = sqrt(mean_squared_error(predictions, actual))
    
    return rmse


In [ ]:
# Get recommendations for the song 'SLovelight' by ABBA
song, artist = 'Strawberry Swing','[\'Coldplay\']'
recommendations = get_recommendations(song,artist, cosine_similarity_pca, merged_df_newer)
print(f"Since you listened to {song}, you might also like:")
for i in recommendations:
    print('Index:', i['Index'], 'Name:', i['Name'], ' By:', i['Artists'])


In [ ]:
# load a dataframe
df_dup = pd.read_csv('merged_df_newer.csv')
# I wanna remove duplicates where name and artists are the same
df_dup.drop_duplicates(subset=['name','artists'], keep='first', inplace=True)
df_dup.drop_duplicates(subset=['name','artists'], keep='first', inplace=True)


In [ ]:
df_dup.to_csv('merged_df_newer.csv', index=False)

In [ ]:
# load a dataframe
new_df = pd.read_csv('merged_df_newer.csv')
# show data where artists name is 'Michael Jackson', show 'name'
print(new_df[new_df['artists'] == '[\'Red Hot Chili Peppers\']']['name'])